In [1]:
#устанавливаем пакеты для чтения и записи файлов Excel

In [2]:
pip install xlrd

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install XlsxWriter

Note: you may need to restart the kernel to use updated packages.


In [4]:
#загружаем необходимые для работы библиотеки
import pandas as pd
import datetime

#загружаем необходимые таблицы из файлов Excel
# таблица с иерархией уровней номенклатуры
levels_goods = pd.read_excel('data/levels_goods.xlsx')

# таблица с историей категорий
history_categories = pd.read_excel('data/History_Categories_Дизельное топливо_RU.xlsx')

In [5]:
# данной строкой можем просмотреть первые 5 записей из таблицы
history_categories.head()

,Период,Код,Номенклатура,Код.1,Категория
0,2015-08-01,RU2667,Дизельное топливо,5,Льготная
1,2015-11-20,RU2912,Дизельное топливо,5,Не льготная
2,2015-11-20,RU2911,Дизельное топливо,5,Не льготная
3,2015-11-20,RU2910,Дизельное топливо,5,Не льготная
4,2015-11-20,RU2909,Дизельное топливо,5,Не льготная


In [6]:
levels_goods.head()

,Наименование,Код,УровеньНоменклатуры_ДТ,Уровень1,Уровень2,Уровень3
0,Дизельное топливо,5,1,5,NaN,NaN
1,ДТ,11,2,5,11.0,NaN
2,ДТ E100 Green 6,364,3,5,11.0,364.0
3,ДТ E100 L-Green 6,17011564,3,5,11.0,17011564.0
4,ДТ GreenPower,127,3,5,11.0,127.0


In [7]:
# удаляем строки содержащие пустые значения NaN
levels_goods = levels_goods.dropna().reset_index()

# коды не должны содержать цифры после запятой, приводим к типу int
levels_goods['Уровень2'] = levels_goods['Уровень2'].astype('int')
levels_goods['Уровень3'] = levels_goods['Уровень3'].astype('int')

# оставляем только необходимые нам столбцы с кодами уровней
levels_goods = levels_goods.drop(['Наименование', 'Код', 'УровеньНоменклатуры_ДТ','index'], axis=1)
levels_goods.head()

,Уровень1,Уровень2,Уровень3
0,5,11,364
1,5,11,17011564
2,5,11,127
3,5,11,124
4,5,11,120


In [8]:
# функция изменяет входящую таблицу с транзакциями
def change_transactions_df(df,levels_goods):   
    fulltransaction = df
    
    # меняем формат времени с 20.06.2019 на стандартный 2019-06-20
    fulltransaction['Дата транзакции'] = pd.to_datetime(fulltransaction['Дата транзакции'],format="%d.%m.%Y")
    
    # присоединяем к таблице транзакций данные по уровняем номенклатуры, присоединение идет по коду
    fulltransaction = fulltransaction.join(levels_goods.set_index('Уровень3'), on='Номенклатура.Код')
    print(fulltransaction.shape)
    
    # удаляем транзакции по которым у нас нет данных по уровням. К примеру если таблица уровней только по ДТ, из транзакционного файла удаляем все кроме ДТ
    fulltransaction = fulltransaction.dropna(subset=['Уровень2']).reset_index()
    fulltransaction = fulltransaction.drop(['index'], axis=1)
    
    # коды не должны содержать цифры после запятой, приводим к типу int
    fulltransaction['Уровень1'] = fulltransaction['Уровень1'].astype('int')
    fulltransaction['Уровень2'] = fulltransaction['Уровень2'].astype('int')
    
    # добавляем столбцы куда будем записывать результаты проверки. первый это найденное значение, второй это результат сравнения с исходным значением
    fulltransaction['Checking_value'] = ''
    fulltransaction['Checking'] = ''
    
    # если категория АЗС не задана, заменяем значения NaN на пустую строку
    fulltransaction['Категория АЗС'] = fulltransaction['Категория АЗС'].fillna('')
    print(fulltransaction.shape)
    return fulltransaction

# функция делает анализирует результаты проверки по уровням и сохраняет результаты в строку
def search_lvl(table_categories, country_table, index, our_row):
    table_AZS = table_categories.loc[table_categories['Код'] == our_row['Код АЗС']] 
    result = ''
    search_lvl3 = search_good(table_AZS,'Номенклатура.Код',our_row)
    if search_lvl3 == False:
        search_lvl2 = search_good(table_AZS,'Уровень2',our_row)
        if search_lvl2 == False:
            search_lvl1 = search_good(table_AZS,'Уровень1',our_row)
            if search_lvl1 == False:
                result = ''
            else:
                result = search_lvl1
        else:
            result = search_lvl2
    else:
        result = search_lvl3 
    country_table.loc[index, 'Checking_value'] = result
    if (our_row['Категория АЗС'] != result):
        country_table.loc[index, 'Checking'] = 'mistake'
    else:
        country_table.loc[index, 'Checking'] = 'OK'
    
# функция проверка по уровню
def search_good(table,level_name,row):
    # из таблицы категорий (отобранной по АЗС) получаем данные по конкретному уровню. 
    # делаем отбор по дате, при удачном поиске возвращаем значение категории   
    result_good = table.loc[table['Код.1'] == row[level_name]]
    if result_good.shape[0] == 0:
        return False
    else:
        result_period = result_good.loc[result_good['Период'] <= row['Дата транзакции']]
        if result_period.shape[0] == 0:
            return False
        else:
            result_period = result_period.sort_values('Период', ascending=False).reset_index().drop(['index'], axis=1)
            result_row = result_period.loc[0]
            return result_row['Категория'] 
        
# функция загружает исходную таблицу, запускает проверку по строкам и сохраняет итоговый результат в файл Excel
def checking_category(name, levels_goods, history_categories):
    table_transaction = pd.read_excel('data/' + name + '.xlsx')
    table_fulltransaction = change_transactions_df(table_transaction,levels_goods)

    for index, row in table_fulltransaction.iterrows():
        if index % 1000 == 0:
            print(index)
            print(str(datetime.datetime.now()))
        search_lvl(history_categories, table_fulltransaction, index, row)

    # Specify a writer
    writer = pd.ExcelWriter('data/' + name + '_result.xlsx', engine='xlsxwriter')
    print('finish ' + name + '_result.xlsx')
    
    # Write your DataFrame to a file     
    table_fulltransaction.to_excel(writer, 'Sheet1')

    # Save the result 
    writer.save()

In [9]:
# запуск проверки по файлу
checking_category('RU_фев19', levels_goods, history_categories)

(50749, 7)
(50749, 9)
0
2020-01-21 07:55:03.518661
1000
2020-01-21 07:55:26.765354
2000
2020-01-21 07:55:50.351628
3000
2020-01-21 07:56:13.714959
4000
2020-01-21 07:56:36.839392


KeyboardInterrupt: 